# Recommendation System TF-IDF

In [111]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [112]:
# import libraries
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle
from sklearn.utils import shuffle

from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [113]:
import os 

In [114]:
os.getcwd()

'C:\\Users\\Matthew Wong\\YHacks'

## Data Cleaning and Feature Eningeering

#### Dont have to run this section again

In [115]:
# Import data
userbase = pd.read_csv(r"data\userdata.csv")
userbase2 = pd.read_csv(r"data\userdata (1).csv")
userbase3 = pd.read_csv(r"data\userdata (2).csv")

In [116]:
dfs = [userbase, userbase2, userbase3]

user_data = pd.concat(dfs)

In [117]:
user_data = user_data.reset_index(drop=True)

In [118]:
user_data.to_csv('user_data.csv')

In [119]:
user_data['size'] = user_data['size'].astype(str)

In [144]:
user_data['NER'] = user_data['countries'].str.cat(user_data[['user identity','size']], sep=',')

In [145]:
user_data

,countries,size,user identity,NER
0,romania,3,refugee,"romania,refugee,3"
1,turkey,1,refugee,"turkey,refugee,1"
2,slovakia,2,support,"slovakia,support,2"
3,slovakia,1,support,"slovakia,support,1"
4,moldova,3,refugee,"moldova,refugee,3"
...,...,...,...,...
2995,romania,5,support,"romania,support,5"
2996,slovakia,5,support,"slovakia,support,5"
2997,slovakia,5,support,"slovakia,support,5"
2998,poland,3,refugee,"poland,refugee,3"


## FEATURE EXTRACTION via tfidf vectorizer
### Create a data-term matrix (vocabulary)

In [164]:
# Instantiate vecorizer object - call tokenize
tf_model = TfidfVectorizer(max_features=200,max_df=0.25,min_df=0.01,stop_words='english')

In [165]:
# create corpus (categories) used for the TF-IDF
corpus = user_data['NER']

In [166]:
tf_matrix = tf_model.fit_transform(corpus)
tf_matrix.shape

(3000, 7)

In [167]:
tf_matrix = tf_model.fit_transform(corpus).todense()
print('The shape of the matrix is:', tf_matrix.shape)
tf_df = pd.DataFrame(tf_matrix)
tf_df.columns = sorted(tf_model.vocabulary_)
display(tf_df.head())

The shape of the matrix is: (3000, 7)


,hungary,moldova,poland,romania,slovakia,turkey,ukraine
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [169]:
# Get cosine similarity of the matrix
cosine_sim = cosine_similarity(tf_matrix, tf_matrix)

# need to run again for a very small portion of the matrix

In [226]:
# Fit on TF-IDF Vectors
nn = NearestNeighbors(n_neighbors=10, algorithm='brute')
nn.fit(tf_matrix)

NearestNeighbors(algorithm='brute', n_neighbors=10)

In [227]:
# Construct a reverse mapping of indices and post titles, and drop duplicate titles if any
indices = pd.Series(user_data.index, index=user_data['NER'])#.drop_duplicates()
print(type(indices))
print(type(cosine_sim))

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [228]:
# Pickle the dtm and tf for use in the prediction
pickle.dump(tf_matrix, open('tf_matrix.pkl', 'wb'))
pickle.dump(tf_model, open('tf_model.pkl', 'wb'))
indices.to_pickle('indices.pkl')
np.save("cosine_sim.npy", cosine_sim)

### Test

In [244]:
# Create the test case
#ideal_category = ['romania','support','1']
ideal_category = ['poland','support','5']

new = tf_model.transform(ideal_category)
results = nn.kneighbors(new.todense())

In [245]:
results

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([[ 531, 2547, 2567, 2563, 1249, 2554, 1244,  527, 2568, 2548],
        [1999, 1997, 2001, 2000, 2003, 1998, 1996, 2005, 1995, 2002],
        [1999, 1997, 2001, 2000, 2003, 1998, 1996, 2005, 1995, 2002]],
       dtype=int64))

In [246]:
# displays the category of the first value of the first array 
user_data['NER'][[results[1][0][0]]]

531    poland,support,1
Name: NER, dtype: object

In [247]:
def give_recommended_TF(ideal_categories):
    ideal_category = [ideal_categories]
    new = tf_model.transform(ideal_category)
    results = nn.kneighbors(new.todense())
    display(user_data['NER'][[results[1][0][0]]])
    display(results)

In [248]:
give_recommended_TF('ukraine,refugee,1')

1929    ukraine,refugee,5
Name: NER, dtype: object

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[1929,  301, 2738, 1032,  512, 1388,  510, 1801,  303, 2743]],
       dtype=int64))